# RSD AI Case Study — Data Visualization & Analysis

This notebook guides you through exploring the synthetic Refugee Status Determination (RSD) dataset to uncover patterns of bias in automated decision-making.

**Structure:**
- 0. Helper Functions
- 1. Import & Preview Data
- 2. Explore Applicant Demographics
- 3. Examine Scoring Functions
- 4. Analyze AI & Final Decisions
- 5. Human Oversight & Override Patterns
- 6. Appeals & Bias Audit
- 7. Build a Predictive Model
- 8. Discussion Questions

## 0. Helper Functions

Run this cell first. It defines all the plotting utilities used throughout the notebook.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

sns.set_theme(style="whitegrid", palette="muted")


def viewDecisionDistribution(data, decision_col='final_decision'):
    """Pie chart of approve vs deny for any decision column."""
    counts = data[decision_col].value_counts()
    colors = ['#66c2a5', '#fc8d62']
    plt.figure()
    plt.pie(counts, labels=counts.index, autopct='%1.1f%%',
            startangle=90, colors=colors)
    plt.title(f'Distribution of {decision_col}')
    plt.tight_layout()
    plt.show()


def viewApprovalRateBy(data, groupby_col, decision_col='final_decision'):
    """Bar chart showing approval rate broken down by any categorical variable."""
    approval_rate = (
        data.groupby(groupby_col)[decision_col]
        .apply(lambda x: (x == 'approve').mean())
        .sort_values()
        .reset_index()
    )
    approval_rate.columns = [groupby_col, 'approval_rate']
    plt.figure(figsize=(8, 4))
    plt.barh(approval_rate[groupby_col], approval_rate['approval_rate'],
             color='#66c2a5', edgecolor='white')
    plt.axvline(x=approval_rate['approval_rate'].mean(), color='#fc8d62',
                linestyle='--', linewidth=1.5, label='Overall average')
    plt.xlabel('Approval Rate')
    plt.title(f'Final Decision Approval Rate by {groupby_col}')
    plt.xlim(0, 1)
    plt.legend()
    plt.tight_layout()
    plt.show()


def viewScoreDistribution(data, score_col, hue_col=None):
    """Histogram of any continuous score, optionally split by a category."""
    plt.figure(figsize=(8, 4))
    if hue_col:
        for val in data[hue_col].unique():
            subset = data[data[hue_col] == val]
            sns.kdeplot(subset[score_col], label=str(val), fill=True, alpha=0.4)
        plt.legend(title=hue_col)
    else:
        sns.histplot(data[score_col], bins=30, kde=True, color='#66c2a5')
    plt.xlabel(score_col)
    plt.ylabel('Density')
    plt.title(f'Distribution of {score_col}' + (f' by {hue_col}' if hue_col else ''))
    plt.tight_layout()
    plt.show()


def viewScoreByDecision(data, score_col, decision_col='final_decision'):
    """Boxplot comparing a score across approve vs deny groups."""
    plt.figure(figsize=(6, 4))
    sns.boxplot(data=data, x=decision_col, y=score_col,
                palette={'approve': '#66c2a5', 'deny': '#fc8d62'})
    plt.title(f'{score_col} by {decision_col}')
    plt.tight_layout()
    plt.show()


def viewCategoricalBreakdown(data, category_col, hue_col='final_decision'):
    """Grouped bar chart for any categorical variable split by a hue."""
    plt.figure(figsize=(9, 4))
    sns.countplot(data=data, x=category_col, hue=hue_col,
                  palette={'approve': '#66c2a5', 'deny': '#fc8d62',
                           'Male': '#66c2a5', 'Female': '#fc8d62', 'Non-binary': '#8da0cb'},
                  order=data[category_col].value_counts().index)
    plt.xticks(rotation=45, ha='right')
    plt.title(f'{category_col} breakdown by {hue_col}')
    plt.tight_layout()
    plt.show()


def plotCorrelationHeatmap(data, cols):
    """Heatmap of correlations between selected numeric columns."""
    plt.figure(figsize=(8, 6))
    corr = data[cols].corr()
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm',
                center=0, linewidths=0.5)
    plt.title('Correlation Matrix')
    plt.tight_layout()
    plt.show()


def plotFeatureImportance(model, X_train):
    """Bar chart of feature importances from a trained Random Forest."""
    importances = pd.Series(model.feature_importances_, index=X_train.columns)
    importances = importances.sort_values(ascending=True)
    plt.figure(figsize=(7, 5))
    importances.plot(kind='barh', color='#66c2a5', edgecolor='white')
    plt.xlabel('Feature Importance')
    plt.title('Feature Importance from Random Forest')
    plt.tight_layout()
    plt.show()

print("Helper functions loaded!")

## 1. Import & Preview Data

In [ ]:
# Load the dataset — update path if running locally
# If using Google Colab, upload the CSV or mount your Drive first
df = pd.read_csv('../dataset/synthetic_RSD_dataset.csv')

print(f"Dataset shape: {df.shape}")
df.head(10)

In [ ]:
# Column types and null counts
df.info()

In [ ]:
# Summary statistics for numeric columns
df.describe().round(2)

## 2. Explore Applicant Demographics

Before looking at decisions, let's understand who is in this dataset.

In [ ]:
# Gender distribution
viewDecisionDistribution(df, decision_col='gender')

In [ ]:
# Country of origin breakdown by gender
viewCategoricalBreakdown(df, 'country_of_origin', hue_col='gender')

In [ ]:
# Education and language proficiency by gender
viewCategoricalBreakdown(df, 'education_level', hue_col='gender')
viewCategoricalBreakdown(df, 'language_proficiency', hue_col='gender')

In [ ]:
# Trauma rate by country of origin
trauma_by_country = (
    df.groupby('country_of_origin')['reported_trauma']
    .mean()
    .sort_values(ascending=False)
)
plt.figure(figsize=(8, 4))
trauma_by_country.plot(kind='bar', color='#fc8d62', edgecolor='white')
plt.ylabel('Proportion Reporting Trauma')
plt.title('Trauma Rate by Country of Origin')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 3. Examine Scoring Functions

The AI uses three scores: credibility, risk, and integration. Let's examine how these scores are distributed and whether they reflect fair or biased patterns.

In [ ]:
# Overall distribution of each score
for score in ['credibility_score', 'risk_score', 'integration_score']:
    viewScoreDistribution(df, score)

In [ ]:
# Does language proficiency inflate credibility? (Intentional bias)
viewScoreDistribution(df, 'credibility_score', hue_col='language_proficiency')

In [ ]:
# Does reported trauma deflate credibility? (Intentional bias)
viewScoreDistribution(df, 'credibility_score', hue_col='reported_trauma')

print("Average credibility score by trauma status:")
print(df.groupby('reported_trauma')['credibility_score'].mean().round(3))

In [ ]:
# Risk score: how many cases hit the 1.0 ceiling?
capped_cases = (df['risk_score'] == 1.0).sum()
print(f"Cases where risk_score was capped at 1.0: {capped_cases}")
print(f"Max uncapped value: {df['risk_score_uncapped'].max():.3f}")

viewScoreDistribution(df, 'risk_score_uncapped')

In [ ]:
# Correlations between scores and key variables
plotCorrelationHeatmap(df, ['credibility_score', 'risk_score',
                             'integration_score', 'state_protection_score', 'age'])

## 4. Analyze AI & Final Decisions

How does the AI decide? Are certain groups approved at higher rates?

In [ ]:
# AI decision vs final decision — did human review change things?
viewDecisionDistribution(df, decision_col='AI_decision')
viewDecisionDistribution(df, decision_col='final_decision')

In [ ]:
# Approval rate by country of origin
viewApprovalRateBy(df, 'country_of_origin')

In [ ]:
# Approval rate by gender
viewApprovalRateBy(df, 'gender')

In [ ]:
# Approval rate by education level and persecution type
viewApprovalRateBy(df, 'education_level')
viewApprovalRateBy(df, 'persecution_type')

In [ ]:
# Score distributions by final decision
for score in ['credibility_score', 'risk_score', 'integration_score']:
    viewScoreByDecision(df, score)

In [ ]:
# Cases denied despite nexus being established — legally suspicious outcomes
nexus_denied = df[(df['nexus_established'] == True) & (df['final_decision'] == 'deny')]
print(f"Cases denied despite nexus established: {len(nexus_denied)}")
print(f"That is {len(nexus_denied)/len(df):.1%} of all cases")
print("\nBreakdown by country:")
print(nexus_denied['country_of_origin'].value_counts())

## 5. Human Oversight & Override Patterns

Only ~10% of cases are reviewed by a human. Does oversight meaningfully correct AI errors?

In [ ]:
# How many cases were reviewed vs overridden?
print(f"Cases reviewed by human:  {df['human_reviewed'].sum()} ({df['human_reviewed'].mean():.1%})")
print(f"Cases overridden:         {df['human_override'].sum()} ({df['human_override'].mean():.1%})")
print(f"Of reviewed cases, overridden: {df[df['human_reviewed']]['human_override'].mean():.1%}")

In [ ]:
# Did overrides help the right people?
overridden = df[df['human_override'] == True]
print("AI decision vs final decision among overridden cases:")
print(overridden[['AI_decision', 'final_decision']].value_counts())

In [ ]:
# Processing time: oversight vs efficiency tradeoff
viewScoreDistribution(df, 'processing_time_days', hue_col='human_reviewed')
print("\nAverage processing time (days):")
print(df.groupby('human_reviewed')['processing_time_days'].mean().round(1))

In [ ]:
# Processing time by final decision
viewScoreByDecision(df, 'processing_time_days')

## 6. Appeals & Bias Audit

In [ ]:
# Who appeals?
denied = df[df['final_decision'] == 'deny']
print(f"Cases that appealed: {df['appealed'].sum()} ({denied['appealed'].mean():.1%} of denied cases)")

In [ ]:
# Appeal outcomes
appeal_counts = df[df['appealed'] == True]['appeal_outcome'].value_counts()
plt.figure(figsize=(5, 4))
plt.pie(appeal_counts, labels=appeal_counts.index, autopct='%1.1f%%',
        colors=['#fc8d62', '#66c2a5'], startangle=90)
plt.title('Appeal Outcomes')
plt.tight_layout()
plt.show()

In [ ]:
# Who gets overturned on appeal?
overturned = df[df['appeal_outcome'] == 'overturned']
print("Country breakdown of overturned appeals:")
print(overturned['country_of_origin'].value_counts())
print("\nGender breakdown of overturned appeals:")
print(overturned['gender'].value_counts())

In [ ]:
# Bias flag distribution
viewCategoricalBreakdown(df, 'bias_flag', hue_col='final_decision')

In [ ]:
# What characterizes severe bias flag cases?
severe = df[df['bias_flag'] == 'severe']
print(f"Severe bias flag cases: {len(severe)}")
print(f"\nTrauma rate in severe cases: {severe['reported_trauma'].mean():.1%}")
print(f"Trauma rate overall:         {df['reported_trauma'].mean():.1%}")
print(f"\nAvg credibility in severe cases: {severe['credibility_score'].mean():.3f}")
print(f"Avg credibility overall:         {df['credibility_score'].mean():.3f}")
print(f"\nNexus established in severe cases: {severe['nexus_established'].mean():.1%}")
print(f"Nexus established overall:         {df['nexus_established'].mean():.1%}")

## 7. Build a Predictive Model (Extension)

Train a Random Forest to predict `final_decision` and examine which features the model considers most important. Compare this to what you found visually above.

In [ ]:
features = df.drop(columns=['id', 'final_decision', 'AI_decision',
                              'human_reviewed', 'human_override',
                              'appealed', 'appeal_outcome', 'bias_flag',
                              'risk_score_uncapped'])
targets = (df['final_decision'] == 'approve').astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    features, targets, test_size=0.3, random_state=42
)

X_train_dum = pd.get_dummies(X_train, drop_first=True)
X_test_dum  = pd.get_dummies(X_test, drop_first=True).reindex(
    columns=X_train_dum.columns, fill_value=0
)

model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train_dum, y_train)

y_pred = model.predict(X_test_dum)
print(f"Model accuracy: {accuracy_score(y_test, y_pred):.3f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
plotFeatureImportance(model, X_train_dum)

## 8. Discussion Questions

Use your analysis above to reflect on the following:

**1. Credibility Bias**
Credibility scores are partly driven by language proficiency and education. Is this a fair proxy for truthfulness? What groups are most disadvantaged by this design?

**2. Trauma Penalty**
Applicants with reported trauma receive a lower credibility score. What does this mean for survivors of sexual violence or detention? Is there a way to correct for this?

**3. Country-of-Origin Patterns**
Which countries have the lowest approval rates? Is the variation driven by risk scores, credibility, nexus rates — or something else?

**4. Human Oversight**
Only ~10% of cases are reviewed by a human, and not all overrides help. Does human review meaningfully correct AI errors, or does it mostly rubber-stamp them?

**5. Appeals as a Safety Net**
40% of appealed cases are overturned. What does this suggest about the accuracy of the original AI decision? Who bears the cost of waiting for an appeal?

**6. Feature Importance**
Look at the Random Forest feature importance chart. Which features drive decisions most? Are any of them proxies for sensitive attributes like gender or country of origin?